# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [40]:
import numpy as np
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
mca = pd.read_csv(url)
mca.head(1)

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.21696,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,NaN


In [60]:
mca.columns = [col.replace(' ', "_").lower() for col in mca.columns]
mca.rename(columns= {'state': 'location'}, inplace=True)
mca.rename(columns= {'unnamed:_0': 'index'}, inplace=True)
mca['location'] = mca['location'].apply(lambda x: 'Arizona' if x == 'AZ' else ('California' if x == 'Cali' else ('Washington' if x == 'WA' else x)))
mca['education'] = mca['education'].apply(lambda x: 'Bachelor' if x == 'Bachelors' else x)


mca['vehicle_class'] = mca['vehicle_class'].apply(lambda x: 'Luxury' if x == 'Sports Car' else ('Luxury' if x == 'Luxury SUV' else ('Luxury' if x == 'Luxury Car' else x)))

mca['gender'] = mca['gender'].map({'female':'female', 'male':'male','F': 'female', 'Femal':'female', 'M':'male', 'Male':'male'})
mca['customer'].fillna('UNKNOWN', inplace=True)
mca['gender'].fillna('UNKNOWN', inplace=True)
mca['location'].fillna('UNKNOWN', inplace=True)
mca['education'].fillna('UNKNOWN', inplace=True)
mca['income'].fillna(mca['income'].mean(),inplace = True)
mca['monthly_premium_auto'].fillna(mca['monthly_premium_auto'].mean(),inplace = True)
mca['policy_type'].fillna('mode', inplace=True)
mca['vehicle_class'].fillna('mode', inplace=True)
mca['total_claim_amount'].fillna(mca['total_claim_amount'].mean(),inplace = True)
#mca["number_of_open_complaints"]= mca["number_of_open_complaints"].dropna().apply(lambda row: str(row).split("/")[1])
mca['number_of_open_complaints'] = mca['number_of_open_complaints'].astype(float)
mca['number_of_open_complaints'].fillna(mca['number_of_open_complaints'].mean(),inplace = True)


mca['customer_lifetime_value'] = mca['customer_lifetime_value'].apply(lambda x: str(x).strip("%"))
mca['customer_lifetime_value'] = mca['customer_lifetime_value'].astype(float)
mca['customer_lifetime_value'].fillna(mca['customer_lifetime_value'].mean(),inplace = True)
mca['customer_lifetime_value'].fillna('UNKNOWN', inplace=True)
mca[['customer_lifetime_value', 'income', 'monthly_premium_auto','number_of_open_complaints', 'total_claim_amount']] = mca[['customer_lifetime_value', 'income', 'monthly_premium_auto','number_of_open_complaints', 'total_claim_amount']].astype(int)

mca['income'].fillna(mca['income'].mean(),inplace = True)
mca['monthly_premium_auto'].fillna(mca['monthly_premium_auto'].mean(),inplace = True)
mca['policy_type'].fillna('mode', inplace=True)
mca['vehicle_class'].fillna('mode', inplace=True)
mca['total_claim_amount'].fillna(mca['total_claim_amount'].mean(),inplace = True)

mca.duplicated(subset=['customer']).sum()
mca.drop_duplicates(subset=["customer"], keep = "first", inplace=True)

mca.duplicated().sum()
mca.reset_index(drop=True, inplace=True)
display(mca.head(1))
mca['number_of_open_complaints'].value_counts()
mca.set_index("index", inplace=True)

,index,customer,location,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809,No,Basic,College,2/18/11,Employed,male,...,0,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [77]:
filtered_mca = mca[(mca['total_claim_amount'] > 1000) & (mca['response'] == 'Yes')]
filtered_mca.head(3)

,customer,location,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
index,,,,,,,,,,,,,,,,,,,,,
189,OK31456,California,11009,Yes,Premium,Bachelor,1/24/11,Employed,female,51643,...,0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358,Luxury,Medsize,NaN
236,YJ16163,Oregon,11009,Yes,Premium,Bachelor,1/24/11,Employed,female,51643,...,0,1,Special Auto,Special L3,Offer2,Agent,1358,Luxury,Medsize,A
419,GW43195,Oregon,25807,Yes,Extended,College,2/13/11,Employed,female,71210,...,1,2,Personal Auto,Personal L2,Offer1,Branch,1027,Luxury,Small,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [79]:
yes = mca[mca['response'] == 'Yes']
average_claim_amount = yes.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2)
display(average_claim_amount)

# The total claim amount for women tends to be bigger in most policy types. In personal auto we observ that men have bigger claim amounts. The personal auto has the highest claim amount for men, while the special auto has the highest claim amount for women.


policy_type     gender
Corporate Auto  female    431.39
                male      409.58
Personal Auto   female    452.67
                male      455.66
Special Auto    female    459.94
                male      414.40
Name: total_claim_amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [105]:
customer_location = mca.groupby(['location'])['customer'].count()
display(customer_location)

more = customer_location[customer_location > 500]
display(more)

location
Arizona       1655
California    3035
Nevada         863
Oregon        2517
UNKNOWN        290
Washington     774
Name: customer, dtype: int64

location
Arizona       1655
California    3035
Nevada         863
Oregon        2517
Washington     774
Name: customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [110]:
#customer_lifetime_value 
cltv_stats = mca.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median'])
display(cltv_stats)


max   min  median
education            gender                     
Bachelor             female  73225  1904  5678.0
                     male    67907  1898  5555.0
College              female  61850  1898  5621.0
                     male    61134  1918  5989.0
Doctor               female  44856  2395  5332.0
                     male    32677  2267  5620.0
High School or Below female  55277  2144  6044.0
                     male    83325  1940  6176.0
Master               female  51016  2417  5801.0
                     male    50568  2272  5617.5

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here